### Label Encoding
* Karşımıza Binary Encoding olarak da gelebilir.
* Eğer bir kategorik değişkenin 2 sınıfı varsa bu 1 ve 0 olarak kodlanırsa buna binary encoding denir.
* Elimizdeki bir kategorik değişken eğer label encodera sokulmuş olursa ve 2'den fazla sınıfı varsa bu durumda label encoding yapılmış olur.
* Label encoding > Binary encoding
* Genel ismi label encodingdir.

In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [69]:
def load():
    data = pd.read_csv("titanic.csv")
    return data
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [70]:
df["Sex"].head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

* Yapmak istediğimiz şey buradaki değişkeni binary encode etmek yani label  encode etmek...
###### Peki neden? Bu nedeni çok iyi kavramak gerekir.
* Kullanılacak olan makine öğrenmesi yöntemlerini zaten kullanmak istediğimizde de bu yöntemler izin vermeyecektir. Çünkü onlar matematiksel gösterimlerle çalışan yöntemlerdir, bunun da bazı istisnaları vardır. Bazı yöntemlerde örneğin kategorik değişkenleri özel bir ifade ediliş tarzıyla ifade edebilirsek bu durumda onların kategorik değişken olduğu anlaşılıp ona göre bir muamele yapılabilir. Ancak genel, yaygın kullanımda bir label encoder kullanmamız one hot encoder kullanmamızın sebebi algoritmaların bizden beklediği bir standart format vardır ve veriyi buna uygun hale getirmemiz gerekir. Tek sebebi bu değildir. Bazen yaptığımız örneğin one hot encoding işlemlerinde amacımız bir kategorik değişkenin önemli olabilecek sınıfını değişkenleştirerek ona bir değer atfetmek olacaktır. Dolayısıyla özetle iki çerçeveden encoding işlemlerini değerlendiriyoruz. 
* Birincisi kullanacak olduğumuz modelleme tekniklerinin bizden beklediği bir standart var.
* İkincisi bu standartla beraber bizim model tahmin performansını iyileştirebileceğimiz, geliştirebileceğimiz de bazı noktalara var, bunlardan dolayı bu işlemleri yapmak istiyoruz. 

In [71]:
# Cinsiyetleri 0 1 olarak kodlamak istediğimizi düşünelim....

# Bunun için LabelEncoder'ı getirmeliyiz:
le = LabelEncoder()
le.fit_transform(df["Sex"])[0:5]

array([1, 0, 0, 0, 1])

##### DİKKAT: Peki bu değiştirme işlemi neye göre gerçekleşiyor?

* LabelEncoder alfabetik sıraya göre ilk gördüğü değere 0 değerini verir.

##### Peki diyelim ki hangisine 0 hangisine 1 verdiğimizi unuttuk, bu durumda ne yapmalıyız?

* inverse_transform ile aşağıdaki gibi yapabiliriz:

In [72]:
le.inverse_transform([0, 1])

array(['female', 'male'], dtype=object)

In [73]:
# Bu aşağıdaki şekilde tercih ediyor olacağız:
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()       # LabelEncoder' ı çağırma işlemi:
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])     # fit_transform işlemi:
    return dataframe

In [74]:
df = load()

* Veri setimize label_encoder fonksiyonunu uygulamak istiyor ama tabii ki yine bir problemimiz var. Nedir bu problem?
* Bir tane değişken olduğunda bunu kolayca uyguladık. Yaygın problemimiz bunu ölçeklenebilir yapıyor olmaktı. Yani elimizde yüzlerce değişken varsa nasıl yapacağız?
* Bu durumda binary colları seçebiliriz. İki seçeneğimiz var:
* DİKKAT: Label encoder uygulamak için birincisi şu anda gördüğümüz bu yöntemi uygulayabiliriz, ikincisi one hot encoder'ı uygulayabiliriz. One hot encoder'ı uygularken get_dummies metodunu kullanırız ve bu get_dummies metodunu kullanırken drop_first'ü eğer True yaparsak bu durumda iki sınıflı kategorik değişkenler de aslında label encoderdan geçirilmiş olunur.
* Sorumuza geri dönecek olursak, elimizde yüzlerce değişken olduğunda neyapacağız? Eeee iki sınıflı kategorik değişkenleri seçmenin bir yolunu bulursak dolayısıyla eğer iki sınıflı kategorik değişkenleri seçmenin yolunu bulabilirsek ve bu iki sınıflı kategorik değişkenleri label encoder'dan geçirirsek ee bu durumda problem çözülür. Onu da aşağıdaki şekilde gerçekleştiriyoruz:

In [75]:
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float] and df[col].nunique() == 2]

* DİKKAT: Diyoruz ki değişkenin sütunlarında gez, gezmiş olduğun bir değişkenin tipine bak, eğer bu değişkenin tipi integer ya da float değilse (bir değişkenin türü integersa ve iki sınıfı varsa zaten bununla ilgilenmiyoruz. Zaten binary encode edilmiş durumda o) ve number unique (eşsiz sınıf sayısı) sayısı 2 olan değişkenleri seç diyoruz. Bakalım kimler varmış:

In [76]:
binary_cols

['Survived', 'Sex']

* df[col].nunique() == 2 kısmını len kullanmadan bu şekilde yapmamızın sebebi unique metodunu kullanıp da len'ini alırsak unique metodu değişkenin içerisindeki eksik değerleri de sınıf olarak görür. Dolayısıyla bunu unique len yaptığımızda ilgili değişkende eksik değer varsa bu durumda len 3 çıkacaktır. Bu sebeple bunu farkında olarak ilgili değişkenin number unique sayısı 2 olsun dedik. Çünkü number unique metodu eksik değeri bir sınıf olarak görmez. 

In [77]:
for col in binary_cols:
    
    # yeniden atama işlemi yapmadık çünkü label_encoder fonksiyonumuzdaki dataframe[binary_col] = labelencoder.fit_transform
    # (dataframe[binary_col]) kısmında atama yaptığımız için burada tekrar atama yapmamız gerekmedi, zaten fonksiyon içinde 
    # kalıcı değişiklik yapıyoruz.
    label_encoder(df, col)

In [78]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


##### Daha büyük bir veri setinde deneyelim:

In [79]:
def load_application_train():
    data = pd.read_csv("application_train.csv")
    return data

In [80]:
df = load_application_train()
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
df.shape

(307511, 122)

In [82]:
# yine benzer şekilde binary_col seçmeliyiz. head atarak bunu yapmak çok zor...
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float]
               and df[col].nunique() == 2]

In [83]:
binary_cols

['TARGET',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EMERGENCYSTATE_MODE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [84]:
df[binary_cols].head()

,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,1,Cash loans,N,Y,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Cash loans,N,N,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Revolving loans,Y,Y,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,Cash loans,N,Y,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,Cash loans,N,Y,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
for col in binary_cols:
    label_encoder(df, col)

* DİKKAT: Yine sinsi bir problemle karşı karşıyayız. EMERGENCYSTATE_MODE değişkeni içerisinde eksik değerler vardı. Tamam biz bunun unique sayısının 2 olduğunu yakaladık başarılı ama label_encoderdan geçirdiğimizde eksik değerleri de doldurmuş. Burada nasıl bir yaklaşım yapmamız lazım? Uçtan uca uygulama yapacak olduğumuz bölümde eksik değerleri daha öncesinden göz önünde bulunduruyor olacağız. Bununla beraber eksik değerleri de böyle dolduruyor olmasını tercih etmek de yaygınca kullanılan bir diğer seçenektir. Sadece burada şunu biliyor olmak lazım biz burada eksik değerlere 2 değerini verdik, bunu biliyor olmak gerekir. Ya da eksik değere sahip gözlemleri uçurmak lazım. Özetle buradaki NA'leri farkında olmak gerekiyor.

In [86]:
# Bahsettiğimiz bu sinsi problemi ele alalım:
df = load()
df["Embarked"].nunique()

3

In [87]:
# Bunu doğrulamak için:
df["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [89]:
df["Embarked"].nunique()

3

In [91]:
# Bize eşsiz değerleri getirir, içinde na de var.
len(df["Embarked"].unique())

4

In [94]:
df["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)